In [1]:
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import pandas as pd

In [2]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 3.9 MB/s eta 0:00:00


In [3]:
import ultralytics

ultralytics.checks()

Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


In [46]:
from ultralytics import YOLO

In [5]:
pip install cvzone

  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26297 sha256=49d712caa7be12c70bf33b44a6df8ae4af55f5f60b055e99fec62251f444be3c
  Stored in directory: /root/.cache/pip/wheels/2c/9f/b3/92e945ac4a71bf727a92463f38155cc5a4fa49c5010b38ec4c
Successfully built cvzone


In [6]:
import cvzone

In [7]:
model = YOLO("yolov8m-seg.pt")

100%|██████████| 52.4M/52.4M [00:00<00:00, 185MB/s]


In [48]:
out = cv2.VideoWriter('/content/output.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 15, (1280,720))
cap = cv2.VideoCapture('/content/People-walking.mp4')

counter1 = []

cy1 = 100
cy2 = 600
offset = 6

while True:
    ret,frame = cap.read()
    if not ret:
        break


    results = model.track(frame)
    segmented_frame = results[0].plot()

    countlist = []
    id_list = []
    for result in results:
      boxes = result.boxes.cpu().numpy()
      boxes_data = boxes.data
      boxes_xyxys = boxes.xyxy
      box_classes = boxes.cls
      seg = result.masks.xy

      for box in boxes:
        box_xyxys = box.xyxy
        cls = box.cls
        if int(cls[0]) == 0:
          x1, y1, x2, y2 = box.xyxy[0]
          x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
          cx = int(x1 + x2) // 2
          cy = int(y1 + y2) // 2
          id = int(box.id[0])

          cv2.circle(frame,(cx,cy),4,(255,0,255),-1)

          if cy>cy1 and cy<cy2:
            cv2.rectangle(frame, (x1,y1) ,(x2,y2), (255,0,0),1)
            cvzone.putTextRect(frame, f'{id}', (x1,y1), 1, 2)

          if counter1.count(id) == 0:
            counter1.append(id)

        if cy<cy1 or cy>cy2:

          cv2.rectangle(frame, (x1,y1) ,(x2,y2), (0,0,255),1)
          cvzone.putTextRect(frame, f'{id}', (x1,y1), 1, 2)

          if counter1.count(id) != 0:
            counter1.remove(id)

      cv2.line(frame,(0,cy1),(1280,cy1),(0,255,0),2)
      cv2.line(frame,(0,cy2),(1280,cy2),(0,255,255),2)

      for xy in seg:
        xy = np.int32([xy])
        cv2.polylines(frame, xy, True, (0,0,255), 4)

      for cls in box_classes:
        if int(cls) == 0:
          countlist.append(int(cls))

    counted = len(countlist)
    p_in = len(counter1)

    cvzone.putTextRect(frame,f'Person in:{p_in}',(50,60),3,1, colorR=(255,0,0))


    out.write(frame)



out.release()
cap.release()
cv2.destroyAllWindows()


0: 384x640 13 persons, 31.1ms
Speed: 2.1ms preprocess, 31.1ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 31.2ms
Speed: 1.9ms preprocess, 31.2ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 persons, 31.1ms
Speed: 1.7ms preprocess, 31.1ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 31.0ms
Speed: 1.9ms preprocess, 31.0ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 27.1ms
Speed: 1.9ms preprocess, 27.1ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 26.8ms
Speed: 1.8ms preprocess, 26.8ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 26.8ms
Speed: 2.1ms preprocess, 26.8ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 26.8ms
Speed: 1.9ms preprocess, 26.8ms inference, 2.8ms postprocess per image at